## Voice Style Control демонстрация

In [1]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import BaseSpeakerTTS, ToneColorConverter

f:\GitProgect\my_openvoice\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



### Инициализация

In [2]:
ckpt_base = 'checkpoints/base_speakers/EN'
ckpt_converter = 'checkpoints/converter'
device="cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs'

base_speaker_tts = BaseSpeakerTTS(f'{ckpt_base}/config.json', device=device)
base_speaker_tts.load_ckpt(f'{ckpt_base}/checkpoint.pth')

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)

f:\GitProgect\my_openvoice\.venv\lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Loaded checkpoint 'checkpoints/base_speakers/EN/checkpoint.pth'
missing/unexpected keys: [] []


f:\GitProgect\my_openvoice\.venv\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\BOSS\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Loaded checkpoint 'checkpoints/converter/checkpoint.pth'
missing/unexpected keys: [] []


### Получение вкраплений тоновых цветов

`source_se` - это вкрапление тонального цвета базового диктора.  Это среднее значение из нескольких предложений, сгенерированных базовым диктором. Мы приводим результат непосредственно здесь, но читатели могут самостоятельно извлечь `source_se`.

In [3]:
source_se = torch.load(f'{ckpt_base}/en_default_se.pth').to(device)

Ссылка `reference_speaker.mp3` ниже указывает на короткий аудиоклип эталона, чей голос мы хотим клонировать. Здесь мы приводим пример. Если вы используете свои собственные эталонные дикторы, пожалуйста, **убедитесь, что каждый диктор имеет уникальное имя файла.** `se_extractor` сохранит `targeted_se`, используя имя файла аудио, и **не будет автоматически перезаписывать.**.

In [4]:
reference_speaker = 'resources/example_reference.mp3' # Это голос, который вы хотите клонировать
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, target_dir='processed', vad=True)

OpenVoice version: v1


f:\GitProgect\my_openvoice\.venv\lib\site-packages\torch\hub.py:293: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to C:\Users\BOSS/.cache\torch\hub\master.zip


[(0.0, 58.8188125)]
after vad: dur = 58.81798185941043


f:\GitProgect\my_openvoice\.venv\lib\site-packages\torch\functional.py:665: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\SpectralOps.cpp:878.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


### Заключение

In [5]:
save_path = f'{output_dir}/output_en_default.wav'

# Запустите базовый динамик tts
text = "This audio is generated by OpenVoice."
src_path = f'{output_dir}/tmp.wav'
base_speaker_tts.tts(text, src_path, speaker='default', language='English', speed=1.0)

# Запустите преобразователь цвета тона
encode_message = "@MyShell"
tone_color_converter.convert(
    audio_src_path=src_path, 
    src_se=source_se, 
    tgt_se=target_se, 
    output_path=save_path,
    message=encode_message)

 > Text splitted to sentences.
This audio is generated by OpenVoice.
 > ===========================
ðɪs ˈɑdiˌoʊ ɪz ˈdʒɛnəɹˌeɪtɪd baɪ ˈoʊpən vɔɪs.
 length:45
 length:45


**Попробуйте разные стили и скорости.** Стиль можно регулировать параметром `speaker в методе `base_speaker_tts.tts`. Доступные варианты: дружелюбный (friendly), веселый(cheerful), взволнованный(excited), грустный(sad), сердитый(angry), испуганный(terrified), кричащий(shouting), шепчущий(whispering). Обратите внимание, что необходимо обновить вставку цвета тона. Скорость можно контролировать с помощью параметра `speed`. Давайте попробуем шепот со скоростью 0,9.

In [6]:
source_se = torch.load(f'{ckpt_base}/en_style_se.pth').to(device)
save_path = f'{output_dir}/output_whispering.wav'

# Запустите базовый динамик tts
text = "This audio is generated by OpenVoice."
src_path = f'{output_dir}/tmp.wav'
base_speaker_tts.tts(text, src_path, speaker='shouting', language='English', speed=0.9)

# Запустите преобразователь цвета тона
encode_message = "@MyShell"
tone_color_converter.convert(
    audio_src_path=src_path, 
    src_se=source_se, 
    tgt_se=target_se, 
    output_path=save_path,
    message=encode_message)

 > Text splitted to sentences.
This audio is generated by OpenVoice.
 > ===========================
ðɪs ˈɑdiˌoʊ ɪz ˈdʒɛnəɹˌeɪtɪd baɪ ˈoʊpən vɔɪs.
 length:45
 length:45


**Попробуйте с разными языками.**  OpenVoice может обеспечить мультиязычное клонирование голоса, просто заменив базовый динамик. Мы приводим здесь пример с китайским базовым диктором и рекомендуем читателям попробовать `demo_part2.ipynb` для детальной демонстрации.

In [ ]:

ckpt_base = 'checkpoints/base_speakers/ZH'
base_speaker_tts = BaseSpeakerTTS(f'{ckpt_base}/config.json', device=device)
base_speaker_tts.load_ckpt(f'{ckpt_base}/checkpoint.pth')

source_se = torch.load(f'{ckpt_base}/zh_default_se.pth').to(device)
save_path = f'{output_dir}/output_chinese.wav'

# Run the base speaker tts
text = "今天天气真好，我们一起出去吃饭吧。"
src_path = f'{output_dir}/tmp.wav'
base_speaker_tts.tts(text, src_path, speaker='default', language='Chinese', speed=1.0)

# Run the tone color converter
encode_message = "@MyShell"
tone_color_converter.convert(
    audio_src_path=src_path, 
    src_se=source_se, 
    tgt_se=target_se, 
    output_path=save_path,
    message=encode_message)

**Техника для добра.** Для тех, кто будет использовать OpenVoice для публичного использования: Мы предлагаем вам добавить водяной знак, чтобы избежать возможного злоупотребления. Пожалуйста, обратитесь к классу ToneColorConverter. **MyShell оставляет за собой возможность определять, сгенерирован ли звук OpenVoice**, независимо от того, добавлен водяной знак или нет.